In [ ]:
import multiprocessing
from multiprocessing.pool import ThreadPool
import numpy as np
from pymoo.core.problem import ElementwiseProblem
from pymoo.core.problem import StarmapParallelization
from pymoo.constraints.as_obj import ConstraintsAsObjective
from pymoo.constraints.as_penalty import ConstraintsAsPenalty
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.algorithms.soo.nonconvex.pso import PSO
from pymoo.optimize import minimize

import optimize as op

class MyProblem(ElementwiseProblem):

    def __init__(self):
        super().__init__(n_var=2,
                         n_obj=2,
                         n_ieq_constr=2,
                         xl=np.array([-2,-2]),
                         xu=np.array([2,2]))

    def _evaluate(self, x, out, *args, **kwargs):
        f1 = 100 * (x[0]**2 + x[1]**2)
        f2 = (x[0]-1)**2 + x[1]**2

        g1 = 2*(x[0]-0.1) * (x[0]-0.9) / 0.18
        g2 = - 20*(x[0]-0.4) * (x[0]-0.6) / 4.8

        out["F"] = [f1, f2]
        out["G"] = [g1, g2]


In [ ]:
opt = op.corgiOptimize()
opt.L = 0.2
opt.T_sw = 0.6
opt.getLiftoffState()
print("Initial Cost", opt.objective(opt.bp_init_guess))


class CorgiProblem(ElementwiseProblem):

    def __init__(self, **kwargs):
        super().__init__(n_var=24,
                         n_obj=1,
                         n_ieq_constr=24,
                         xl=np.array([0.03, 0.00, 0.01, -0.05, 0.01, -0.05,
                                      0.03, 0.00, 0.01, -0.05, 0.01, -0.05,
                                      0.03, 0.00, 0.01, -0.05, 0.01, -0.05,
                                      0.03, 0.00, 0.01, -0.05, 0.01, -0.05]),
                         xu=np.array([0.04, 0.02, 1, 0.5, 1, 0.5,
                                      0.04, 0.02, 1, 0.5, 1, 0.5,
                                      0.04, 0.02, 1, 0.5, 1, 0.5,
                                      0.04, 0.02, 1, 0.5, 1, 0.5]),
                         **kwargs)

    def _evaluate(self, x, out, *args, **kwargs):
        f  = opt.objective(x)

        g1 = -1 * opt.innerWsPFConstraint(x)
        g2 = -1 * opt.outerWsPFConstraint(x)
        g3 = np.abs(opt.velocityConstraint(x)) - 0.01
        print("g3", np.average(g3))
        g = np.hstack((g1, g2))
        g = np.hstack((g, g3))

        out["F"] = [f]
        out["G"] = list(g)

# initialize the thread pool and create the runner
# n_proccess = 8
# pool = multiprocessing.Pool(n_proccess)
# runner = StarmapParallelization(pool.starmap)

# initialize the thread pool and create the runner
n_threads = 16
pool = ThreadPool(n_threads)
runner = StarmapParallelization(pool.starmap)
problem = CorgiProblem(elementwise_runner=runner, elementwise_evaluation=True)
# problem = CorgiProblem()


In [ ]:
algorithm = NSGA2(
    pop_size=60,
    n_offsprings=30,
    sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(eta=20),
    eliminate_duplicates=True
)
# algorithm = PSO()

In [ ]:
from pymoo.termination import get_termination

termination = get_termination("n_gen", 100)

In [ ]:

result = minimize(ConstraintsAsPenalty(problem, 1.0),
                  algorithm,
                  termination,
                  seed=1,
                  save_history=False,
                  verbose=False)

pool.close()


In [ ]:
print(result.X)
print(result.F)
min_ = np.min(result.F, axis=0)
opt.exportBezierData(result.X)

In [ ]:
opt = op.corgiOptimize()
opt.L = 0.2
opt.T_sw = 0.6
opt.getLiftoffState()
print("Initial Cost", opt.objective(result.X))